# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [2]:
import graphlab
import numpy

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1500426730.log


This non-commercial license of GraphLab Create for academic use is assigned to bgj2@hi.is and will expire on July 06, 2018.


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.030287     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [8]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    pred = model.predict(data)
    # Then compute the residuals/errors
    error = outcome - pred
    # Then square and add them up
    RS = error*error
    RSS = RS.sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [9]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [10]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [11]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [12]:
# create the remaining 3 features in both TEST and TRAIN data

train_data['bed_bath_rooms'] = train_data['bedrooms', 'bathrooms'].apply(lambda row: row['bedrooms']*row['bathrooms'])
test_data['bed_bath_rooms'] = test_data['bedrooms', 'bathrooms'].apply(lambda row: row['bedrooms']*row['bathrooms'])

In [13]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [14]:
train_data['lat_plus_long'] = train_data['lat', 'long'].apply(lambda row: row['lat']+row['long'])
test_data['lat_plus_long'] = test_data['lat', 'long'].apply(lambda row: row['lat']+row['long'])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [19]:
chosen_features = ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']
for feature in chosen_features:
    mean = test_data[feature].mean()
    print 'The mean of', feature, 'is:', mean

The mean of bedrooms_squared is: 12.4466777016
The mean of bed_bath_rooms is: 7.50390163159
The mean of log_sqft_living is: 7.55027467965
The mean of lat_plus_long is: -74.6533349722


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [20]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [23]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, features = model_1_features, target='price')
model_2 = graphlab.linear_regression.create(train_data, features = model_2_features, target='price')
model_3 = graphlab.linear_regression.create(train_data, features = model_3_features, target='price')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 16492

Number of features          : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.037350     | 3176798.439200     | 2081963.375820       | 227339.891759 | 244342.769789   |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

In [28]:
# Examine/extract each model's coefficients:
model_1_weights = model_1.get('coefficients')
model_2_weights = model_2.get('coefficients')
model_3_weights = model_3.get('coefficients')

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 7

Data:
+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -53666416.9031 | 1669091.32308 |
|  sqft_living   |  None | 301.822208969  | 3.23914500603 |
|    bedrooms    |  None | -109732.607825 | 4837.90453319 |
|   bathrooms    |  None | -68906.8402195 | 7620.84733028 |
|      lat       |  None | 624182.344989  | 13220.3092086 |
|      long      |  None | -198230.636898 | 13418.4513142 |
| bed_bath_rooms |  None | 24073.5120026  | 1968.77836858 |
+----------------+-------+----------------+---------------+
[7 rows x 4 columns]

In [29]:
model_1_weights

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 6

Data:
+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -55653375.845  | 1689523.14578 |
| sqft_living |  None | 310.058204555  | 3.26527411777 |
|   bedrooms  |  None | -60324.3427157 | 2557.61454113 |
|  bathrooms  |  None | 13697.4853094  | 3679.53967427 |
|     lat     |  None | 625700.150299  | 13469.3029068 |
|     long    |  None | -212448.101283 | 13589.2454905 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

In [30]:
model_2_weights

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 7

Data:
+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -53666416.9031 | 1669091.32308 |
|  sqft_living   |  None | 301.822208969  | 3.23914500603 |
|    bedrooms    |  None | -109732.607825 | 4837.90453319 |
|   bathrooms    |  None | -68906.8402195 | 7620.84733028 |
|      lat       |  None | 624182.344989  | 13220.3092086 |
|      long      |  None | -198230.636898 | 13418.4513142 |
| bed_bath_rooms |  None | 24073.5120026  | 1968.77836858 |
+----------------+-------+----------------+---------------+
[7 rows x 4 columns]

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [32]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1_rss_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
model_1_rss_train

971618079290704.0

In [33]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_2_rss_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
model_2_rss_train

962284282608425.2

In [34]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_3_rss_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
model_3_rss_train

905506211582988.2

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [37]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_1_rss_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
model_1_rss_test

226743008758734.56

In [38]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_2_rss_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
model_2_rss_test

224429218143489.94

In [39]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_3_rss_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
model_3_rss_test

249776136606880.94

#### Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.